In [ ]:
# Jupyter Notebook - 代码

# 导入必要的库
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
import datetime
import tensorflow_model_optimization as tfmot
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_apply = tfmot.quantization.keras.quantize_apply
# 设定日志级别
tf.get_logger().setLevel('ERROR')

# 🔹 超参数
IMG_SIZE = (96, 96)
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# 🔹 数据集路径
base_dir = './dataset'
train_dir = os.path.join(base_dir, 'COVER_au')
valid_dir = os.path.join(base_dir, 'COVER_au')

BATCH_SIZE = 128
# 🔹 加载数据集
train_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, 
    validation_split=0.8, 
    subset="training", 
    seed=12,
    batch_size=BATCH_SIZE, 
    image_size=IMG_SIZE)

validation_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir, 
    validation_split=0.2, 
    subset="validation", 
    seed=12,
    batch_size=BATCH_SIZE, 
    image_size=IMG_SIZE)

class_names = train_dataset_raw.class_names
print("Class Names:", class_names)

def preprocess_image(image, label):
    image = tf.cast(image, tf.float32)  # 转换为 float32
    image = image / 255.0              # 归一化到 [0,1]
    return image, label

# 加载数据集
train_dataset = (train_dataset_raw
                 .map(preprocess_image, num_parallel_calls=AUTOTUNE)
                 .cache('train_cache')  # 缓存到文件
                 .shuffle(1000)
                 .prefetch(AUTOTUNE))

validation_dataset = (validation_dataset_raw
                      .map(preprocess_image, num_parallel_calls=AUTOTUNE)
                      .cache('val_cache')
                      .prefetch(AUTOTUNE))


In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    pooling='avg',
    alpha=0.35,
    weights='imagenet')

# 量化注释并应用
annotated_model = tf.keras.Sequential([
    base_model,
    quantize_annotate_layer(tf.keras.layers.Dense(len(class_names), activation='softmax'))
])

# 应用量化
qat_model = quantize_apply(annotated_model)
qat_model.build((None, 96, 96, 3))
qat_model.summary()

In [ ]:
# 编译模型
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001, decay_steps=1000, decay_rate=0.90, staircase=True)

qat_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# 训练第一阶段
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history = qat_model.fit(train_dataset,
                    validation_data=validation_dataset,
                    epochs=100, 
                    callbacks=[early_stopping]
                    )


In [ ]:
# 🔹 直接导出为TFLite格式 (无需保存H5)
def representative_dataset():
    # 从验证集取500个批次作为量化校准数据
    for images, _ in tqdm(validation_dataset.take(500), desc="Calibration"):
        yield [tf.cast(images, tf.float32)]  # 输入需为浮点型

converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8   # 输入为uint8 (0-255)
converter.inference_output_type = tf.uint8  # 输出为uint8类别索引

tflite_model = converter.convert()

output_dir = './model'
os.makedirs(output_dir, exist_ok=True)  # 自动创建目录（如果不存在）
# 保存带时间戳的TFLite模型
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
output_path = f'./model/qat_model_{timestamp}.tflite'
with open(output_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite模型已保存至: {output_path}")

target_dir = "model"
# 直接匹配当前目录下的 .h5 文件
for file in os.listdir(target_dir):
    if file.endswith(".h5"):
        file_path = os.path.join(target_dir, file)
        try:
            os.remove(file_path)
            print(f"已删除: {file_path}")
        except Exception as e:
            print(f"删除失败 [{file_path}]: {e}")